In [ ]:
import time
import glob
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import ipywidgets as pyw
import tkinter.filedialog as tkf
import tifffile
from tabulate import tabulate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import linalg as LA
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid

In [ ]:
# create a customized colorbar
color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))
custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

def zero_one_rescale(spectrum):
    """
    normalize one spectrum from 0.0 to 1.0
    """
    spectrum = spectrum.clip(min=0.0)
    min_val = np.min(spectrum)
    
    rescaled = spectrum - min_val
    
    if np.max(rescaled) != 0:
        rescaled = rescaled / np.max(rescaled)
    
    return rescaled

def fourd_roll_axis(stack):
    stack = np.rollaxis(np.rollaxis(stack, 2, 0), 3, 1)
    return stack

def reshape_coeff(coeffs, new_shape):
    """
    reshape a coefficient matrix to restore the original scanning shapes.
    """
    coeff_reshape = []
    for i in range(len(new_shape)):
        temp = coeffs[:int(new_shape[i, 0]*new_shape[i, 1]), :]
        coeffs = np.delete(coeffs, range(int(new_shape[i, 0]*new_shape[i, 1])), axis=0)
        temp = np.reshape(temp, (new_shape[i, 0], new_shape[i, 1], -1))
        #print(temp.shape)
        coeff_reshape.append(temp)
        
    return coeff_reshape

def radial_indices(shape, radial_range, center=None):
    y, x = np.indices(shape)
    if not center:
        center = np.array([(y.max()-y.min())/2.0, (x.max()-x.min())/2.0])
    
    r = np.hypot(y - center[0], x - center[1])
    ri = np.ones(r.shape)
    
    if len(np.unique(radial_range)) > 1:
        ri[np.where(r <= radial_range[0])] = 0
        ri[np.where(r > radial_range[1])] = 0
        
    else:
        r = np.round(r)
        ri[np.where(r != round(radial_range[0]))] = 0
    
    return ri

In [ ]:
n_split = 2
n_each_split = 5

In [ ]:
split_adr = []
for i in range(n_split):
    directory_path = tkf.askdirectory()
    split_adr.append(glob.glob(directory_path+"/*"))
print(len(split_adr))
split_adr = np.asarray(split_adr)

In [ ]:
file_adr = []
for i in range(n_split):
    ri = np.random.choice(len(split_adr[i]), n_each_split, replace=False)
    file_adr.extend(split_adr[i, ri].tolist())
print(len(file_adr))
print(*file_adr, sep="/n")

In [ ]:
num_img = len(file_adr)
print(num_img)

In [ ]:
data_original = []
data_shape = []
for i in range(num_img):
    tmp = tifffile.imread(file_adr[i])
    #tmp = hys.load(file_adr[i]).data
    #tmp = fourd_roll_axis(tmp)
    print(tmp.shape)
    data_shape.append(list(tmp.shape[:2]))
    data_original.append(tmp)
    
data_shape = np.asarray(data_shape)

In [ ]:
w_size = data_original[0].shape[2]
print(w_size)

dataset_flat = []
for i in range(num_img):
    dataset_flat.extend(data_original[i].reshape(-1, w_size, w_size).tolist())
    
dataset_flat = np.asarray(dataset_flat)
dataset_flat = dataset_flat / np.max(dataset_flat, axis=(1, 2))[:, np.newaxis, np.newaxis]
print(dataset_flat.shape)

In [ ]:
total_num = len(dataset_flat)
ri = np.random.choice(total_num, total_num, replace=False)

dataset_input = dataset_flat[ri]

In [ ]:
class simple_VAE(nn.Module):
    def __init__(self, in_dim, final_length, channels, kernels, strides, paddings, z_dim, 
                 dec_kernels, dec_strides, dec_paddings, dec_outpads, f_kernel):
        super(simple_VAE, self).__init__()
        
        self.in_dim = in_dim
        self.z_dim = z_dim
        self.final_length = final_length
        self.channels = channels
        
        enc_net = []
        enc_net.append(nn.Conv2d(1, channels[0], kernels[0], stride=strides[0], 
                                 padding=paddings[0], bias=True))
        enc_net.append(nn.BatchNorm2d(channels[0]))
        enc_net.append(nn.LeakyReLU(0.1))
        for i in range(1, len(channels)):
            enc_net.append(nn.Conv2d(channels[i-1], channels[i], kernels[i], stride=strides[i],
                                     padding=paddings[i], bias=True))
            enc_net.append(nn.BatchNorm2d(channels[i]))
            enc_net.append(nn.LeakyReLU(0.1))
            
        enc_net.append(nn.Flatten())
        enc_net.append(nn.Linear(self.final_length**2*channels[-1], 2*self.z_dim))
        
        self.encoder = nn.Sequential(*enc_net)
        
        self.init_decoder = nn.Linear(self.z_dim, self.final_length**2*channels[-1])
        
        dec_net = []
        for i in range(len(channels)-1, 0, -1):
            dec_net.append(nn.ConvTranspose2d(channels[i], channels[i-1], dec_kernels[i], dec_strides[i],
                                              dec_paddings[i], output_padding=dec_outpad[i], bias=True))
            dec_net.append(nn.BatchNorm2d(channels[i-1]))
            dec_net.append(nn.LeakyReLU(0.1))
            
        dec_net.append(nn.ConvTranspose2d(channels[0], 1, dec_kernels[0], dec_strides[0], 
                                          dec_paddings[0], output_padding=dec_outpad[0], bias=True))
        dec_net.append(nn.BatchNorm2d(1))
        dec_net.append(nn.LeakyReLU(0.1))
        
        dec_net.append(nn.Conv2d(1, 1, f_kernel, bias=True))
        dec_net.append(nn.Sigmoid())
        
        self.decoder = nn.Sequential(*dec_net)
        
        
    def encode(self, x):
        
        latent = self.encoder(x)
        mu = latent[:, :self.z_dim]
        logvar = latent[:, self.z_dim:]
        
        return mu, logvar
        
                
    def reparametrization(self, mu, logvar):
        
        return mu+torch.exp(0.5*logvar)*torch.randn_like(logvar)
        
        
    def decode(self, z):
        
        init_decoded = self.init_decoder(z)
        init_decoded = init_decoded.view(-1, self.channels[-1], self.final_length, self.final_length)
        
        return self.decoder(init_decoded)
        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        mu, logvar = self.encode(x)
        z = self.reparametrization(mu, logvar)
        
        return mu, logvar, z, self.decode(z)

In [ ]:
if torch.cuda.is_available():
    print("%d gpu available"%(torch.cuda.device_count()))
    cuda_device = torch.device("cuda:0")
    print(torch.cuda.get_device_name(cuda_device))
    torch.cuda.set_device(cuda_device)
else:
    cuda_device = None

In [ ]:
num_comp = 2
channels = [256, 128, 64, 32]
kernels = [10, 10, 10, 10]
padding = [1, 1, 1, 1]
stride = [2, 2, 2, 2]

dat_dim = []
tmp_dim = w_size
for i in range(len(kernels)):
    tmp_dim += (-kernels[i]+2*padding[i])
    tmp_dim /= stride[i]
    tmp_dim += 1
    dat_dim.append(int(tmp_dim))
    #dat_dim.append(tmp_dim)

print(dat_dim)
print(kernels)
print(channels)
print(padding)
print(stride)

In [ ]:
dec_kernel = [10, 10, 10, 10]
dec_stride = [2, 2, 2, 2]
dec_padding = [1, 1, 1, 1]
dec_outpad = [1, 1, 1, 1]

dec_dim = []
enc_dim = dat_dim[-1]
for i in range(len(dec_kernel)):
    enc_dim = (enc_dim-1)*dec_stride[i] + dec_kernel[i] - 2*dec_padding[i] + dec_outpad[i]
    dec_dim.append(enc_dim)
    
print(dec_dim)

dec_kernel.reverse()
dec_stride.reverse()
dec_padding.reverse()
dec_outpad.reverse()

final_kernel = 10
print(dec_dim[-1] - final_kernel + 1)

In [ ]:
model = simple_VAE(w_size, dat_dim[-1], channels, kernels, stride, padding, num_comp, 
                   dec_kernel, dec_stride, dec_padding, dec_outpad, final_kernel)
model = nn.DataParallel(model)
model.cuda(cuda_device)
print(model)

In [ ]:
batch_size = 256
mini_batches = [dataset_input[k:k+batch_size] for k in range(0, len(dataset_input), batch_size)]
print(len(mini_batches))

In [ ]:
n_epoch = 500
l_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=l_rate)
print(optimizer)

In [ ]:
start = time.time()
latent_z = []
z_mu = []
z_logvar = []
n_fig = 5
for epoch in range(n_epoch):
    loss_epoch = 0
    recon_loss = 0
    KLD_loss = 0
    for i, m_batch in enumerate(mini_batches):
        
        x = torch.from_numpy(mini_batches[i])
        x = x.to(torch.float32)
        x = x.to(cuda_device)
        x.requires_grad_(requires_grad=False)
        
        mu, logvar, z, x_ = model(x)
        
        reconstruction_error = F.binary_cross_entropy(x_.squeeze(), x, reduction="sum")
        KL_divergence = -0.5*torch.sum(1+logvar-mu**2-logvar.exp())
        
        loss = reconstruction_error + KL_divergence
        loss_epoch += loss.item()
        recon_loss += reconstruction_error.item()
        KLD_loss += KL_divergence.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch == n_epoch-1:
            latent_z.extend(z.data.cpu().numpy().tolist())
            z_mu.extend(mu.data.cpu().numpy().tolist())
            z_logvar.extend(logvar.data.cpu().numpy().tolist())
            
            
    if epoch == 0:
        print(torch.cuda.memory_summary(device=cuda_device))
        
    if (epoch+1) % int(n_epoch/10) == 0:
        print(tabulate([
                        ["epoch", epoch+1], 
                        ["total loss", loss_epoch/total_num],
                        ["reconstruction error", recon_loss/total_num],
                        ["KL divergence", KLD_loss/total_num],
                        ["error ratio", reconstruction_error/KL_divergence],
                        ]))
        print("%.2f minutes have passed"%((time.time()-start)/60))
        
        fig, ax = plt.subplots(2, n_fig, figsize=(5*n_fig, 5*2))
        for i in range(n_fig):
            ax[0][i].imshow(x[i].data.cpu(), cmap="inferno")
            ax[1][i].imshow(x_[i].squeeze().data.cpu(), cmap="inferno")
        fig.tight_layout()
        plt.show()

latent_z = np.asarray(latent_z)
z_mu = np.asarray(z_mu)
z_logvar = np.asarray(z_logvar)
print("The training has been finished.")

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib qt

In [ ]:
coeffs = np.zeros_like(latent_z)
coeffs[ri] = latent_z.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape)

fig, ax = plt.subplots(1, 1)
ax.grid()
for i in range(num_comp):
    ax.hist(coeffs[:, i], bins=50, alpha=(1.0-i*(1/num_comp)))
plt.show()

nrow, ncol = n_split, n_each_split
print(nrow, ncol)

for j in range(num_comp):
    fig, ax = plt.subplots(nrow, ncol, figsize=(5*ncol, 5*nrow))
    fig.suptitle("latent variable %d"%(j+1), fontsize=20)
    for k, a in enumerate(ax.flat):
        a.imshow(coeffs_reshape[k][:, :, j], cmap="inferno")
        a.axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
coeffs = np.zeros_like(z_mu)
coeffs[ri] = z_mu.copy()
coeffs_reshape = reshape_coeff(coeffs, data_shape)

fig, ax = plt.subplots(1, 1)
ax.grid()
for i in range(num_comp):
    ax.hist(coeffs[:, i], bins=50, alpha=(1.0-i*(1/num_comp)))
plt.show()

nrow, ncol = n_split, n_each_split
print(nrow, ncol)

for j in range(num_comp):
    fig, ax = plt.subplots(nrow, ncol, figsize=(5*ncol, 5*nrow))
    fig.suptitle("latent variable %d"%(j+1), fontsize=20)
    for k, a in enumerate(ax.flat):
        a.imshow(coeffs_reshape[k][:, :, j], cmap="inferno")
        a.axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
coeffs = np.zeros_like(z_logvar)
coeffs[ri] = np.exp(0.5*z_logvar)
coeffs_reshape = reshape_coeff(coeffs, data_shape)

fig, ax = plt.subplots(1, 1)
ax.grid()
for i in range(num_comp):
    ax.hist(coeffs[:, i], bins=50, alpha=(1.0-i*(1/num_comp)))
plt.show()

nrow, ncol = n_split, n_each_split
print(nrow, ncol)

for j in range(num_comp):
    fig, ax = plt.subplots(nrow, ncol, figsize=(5*ncol, 5*nrow))
    fig.suptitle("latent variable %d"%(j+1), fontsize=20)
    for k, a in enumerate(ax.flat):
        a.imshow(coeffs_reshape[k][:, :, j], cmap="inferno")
        a.axis("off")
    fig.tight_layout()
    plt.show()

In [ ]:
tifffile.imsave("latent_variable_maps.tif", coeffs_reshape)

In [ ]:
n_sample = 20
sigma = 3.0
z_test = np.linspace(-sigma, sigma, n_sample*10, endpoint=True)
rv = stats.norm(0, 1)
norm_pdf = rv.pdf(z_test)
norm_pdf = norm_pdf / np.sum(norm_pdf)
z_test = np.sort(np.random.choice(z_test, n_sample, replace=False, p=norm_pdf))
z_test = np.meshgrid(z_test, z_test)
z_test = np.stack((z_test[0].flatten(), z_test[1].flatten()), axis=1)
print(z_test.shape)
z_test = torch.from_numpy(z_test).to(torch.float32).to(cuda_device)

In [ ]:
model.eval()
generated = model.module.decode(z_test)
print(generated.shape)

In [ ]:
fig, ax = plt.subplots(n_sample, n_sample, figsize=(30, 30))
for i, a in enumerate(ax.flat):
    a.imshow(generated[i].squeeze().data.cpu(), cmap="inferno")
    a.axis("off")
plt.subplots_adjust(hspace=0.01, wspace=0.01)
plt.show()

In [ ]:
fig, ax = plt.subplots(n_sample, n_sample, figsize=(30, 30))
for i, a in enumerate(ax.flat):
    tmp = generated[i].squeeze().data.cpu()
    tmp = np.log(np.abs(np.fft.fftshift(np.fft.fft2(tmp))))
    a.imshow(tmp, cmap="inferno")
    a.axis("off")
plt.subplots_adjust(hspace=0.01, wspace=0.01)
plt.show()

In [ ]:
# 2D subspace
%matplotlib qt
fig, ax = plt.subplots(1, 1, figsize=(7, 7))

def projection(c1, c2):
    ax.cla()
    ax.scatter(coeffs[:, c1], coeffs[:, c2], s=30, c="black", alpha=0.5)
    ax.grid()
    ax.set_xlabel("loading vector %d"%(c1+1), fontsize=15)
    ax.set_ylabel("loading vector %d"%(c2+1), fontsize=15)
    ax.tick_params(axis="both", labelsize=15)
    fig.canvas.draw()
    fig.tight_layout()

x_widget = pyw.IntSlider(min=0, max=num_comp-1, step=1, value=0)
y_widget = pyw.IntSlider(min=0, max=num_comp-1, step=1, value=1)

pyw.interact(projection, c1=x_widget, c2=y_widget)
plt.show()